In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np

from sklearn.neighbors import NearestNeighbors

import pickle

In [2]:
df = pd.read_csv("./food_recommendation.csv")
df.head()

,name,category,spice,Aamchur,Ajwain,Almond,Avocados,Bacon,Baking Soda,Banana,...,Tutti Frutti,Urad Dal,Vanilla,Vegetables,Vinegar,Vodka,Wasabi,Wheat,Yeast,Yogurt
0,Aloo Paratha,1,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,Appam,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
2,Bacon,2,7,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Barfi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bengan Bharta,1,7,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Columns: 130 entries, name to Yogurt
dtypes: int64(129), object(1)
memory usage: 110.8+ KB


In [4]:
df.describe()

,category,spice,Aamchur,Ajwain,Almond,Avocados,Bacon,Baking Soda,Banana,Bay Leaves,...,Tutti Frutti,Urad Dal,Vanilla,Vegetables,Vinegar,Vodka,Wasabi,Wheat,Yeast,Yogurt
count,109.000000,109.00000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,...,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000
mean,0.963303,4.00000,0.036697,0.018349,0.027523,0.009174,0.009174,0.128440,0.009174,0.036697,...,0.009174,0.045872,0.018349,0.009174,0.082569,0.009174,0.009174,0.045872,0.036697,0.045872
std,0.637219,2.61052,0.188886,0.134829,0.164357,0.095783,0.095783,0.336125,0.095783,0.188886,...,0.095783,0.210173,0.134829,0.095783,0.276501,0.095783,0.095783,0.210173,0.188886,0.210173
min,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,5.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,6.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.000000,9.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
names = df["name"].copy()
names

0           Aloo Paratha
1                  Appam
2                  Bacon
3                  Barfi
4          Bengan Bharta
             ...        
104     Veg Spring Rolls
105     Vegetable Pakora
106       Vegetable Soup
107              Waffles
108    White Sauce Pasta
Name: name, Length: 109, dtype: object

In [6]:
features = df.drop(["name"], axis=1)
features

,category,spice,Aamchur,Ajwain,Almond,Avocados,Bacon,Baking Soda,Banana,Bay Leaves,...,Tutti Frutti,Urad Dal,Vanilla,Vegetables,Vinegar,Vodka,Wasabi,Wheat,Yeast,Yogurt
0,1,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0
2,2,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,1,5,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
105,1,6,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
106,1,5,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
107,2,2,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
model = NearestNeighbors(metric="euclidean")

model.fit(features)

NearestNeighbors(metric='euclidean')

In [18]:
test = features.iloc[47]
print(test)

category    0
spice       5
Aamchur     0
Ajwain      0
Almond      0
           ..
Vodka       0
Wasabi      0
Wheat       0
Yeast       0
Yogurt      0
Name: 47, Length: 129, dtype: int64


In [19]:
print(names[47])

Jain Pav Bhaji


In [40]:
test = [0] * 127
test.insert(0, 6)
test.insert(0, 0)

In [41]:
dist, ind = model.kneighbors([test], n_neighbors=6)

print(dist, ind)

for i in ind[0][1:]:
    print(names[i])

[[2.         2.23606798 2.23606798 2.44948974 2.44948974 2.44948974]] [[ 47 101   7  36  63  68]]
Veg Pav Bhaji
Bhindi Masala
French Fries
Nachos
Omelet


In [12]:
# pickle.dump(model, open("food_recommendation.pkl", 'wb'))

In [13]:
loaded_model = pickle.load(open("food_recommendation.pkl", 'rb'))

In [14]:
test = [0.2] * 127
test.insert(0, 3)
test.insert(0, 0)

In [15]:
dist, ind = loaded_model.kneighbors([test], n_neighbors=11)

print(dist, ind)

for i in ind[0][1:]:
    print(names[i])

[[0.19665293 0.19990531 0.19990531 0.20149579 0.20497768 0.20837092
  0.20837092 0.20894348 0.20894348 0.20915183 0.20915183]] [[ 44  94 100  61  69  99  98  49   4  59  42]]
Veg Biryani
Veg Kabab
Momos
Onion Rings
Veg Hot Dog
Veg Hakka Noodles
Kadai Paneer
Bengan Bharta
Masala Dosa
Hara Bhara Kabab
